In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
from src.generate_batchs import train_val_batchloader, MRIDataset
from src import config
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

import os


In [19]:
data_train, data_validation= train_val_batchloader(config.DATASET_ROOT_PATH)

In [21]:
for batch in data_train:
    print((batch[0].shape))
    break

torch.Size([2, 3, 240, 240, 155])


In [22]:
dataset = MRIDataset(config.DATASET_ROOT_PATH)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=dataset.collate_fn) #collate_fn=dataset.collate_fn

train_features, train_labels = next(iter(dataloader))


In [23]:
print(train_features.shape)
print(train_labels.shape)

torch.Size([320, 3, 60, 60, 31])
torch.Size([320, 3, 60, 60, 31])


In [19]:
for batch in dataloader:
    print((batch[1].shape))
    break

torch.Size([4, 1, 240, 240, 155])


In [36]:
# Definir el modelo U-Net
class Unet3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Unet3D, self).__init__()

        # Codificador (downsampling)
        self.encoder1 = nn.Sequential(
            nn.Conv3d(in_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )

        self.encoder2 = nn.Sequential(
            nn.MaxPool3d(kernel_size=2, stride=2),
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True)
        )

        # Decodificador (upsampling)
        self.decoder1 = nn.Sequential(
            nn.Conv3d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )

        self.decoder2 = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, out_channels, kernel_size=1),
            nn.Softmax(dim=1)  # Salida con múltiples canales, usar Softmax
        )

    def forward(self, x):
        # Codificación
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)

        # Decodificación
        dec1 = self.decoder1(enc2)
        dec2 = self.decoder2(torch.cat([dec1, enc1], dim=1))

        return dec2

# Crea una instancia del modelo
model = Unet3D(in_channels=3, out_channels=4) 


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Verificar si hay GPU disponible

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Bucle de entrenamiento y validación
num_epochs = 1

for epoch in range(num_epochs):
    # Fase de entrenamiento
    model.train()
    for inputs, labels in data_train:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Paso de adelante y cálculo de la pérdida
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Retropropagación y actualización de los pesos
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Fase de validación
    model.eval()
    with torch.no_grad():
        validation_loss = 0.0
        for inputs_val, labels_val in data_validation:
            inputs_val = inputs_val.to(device)
            labels_val = labels_val.to(device)

            # Paso de adelante y cálculo de la pérdida
            outputs_val = model(inputs_val)
            loss_val = criterion(outputs_val, labels_val)
            validation_loss += loss_val.item()

        # Calcular la pérdida promedio en la validación
        validation_loss /= len(data_validation)

    # Imprimir métricas de entrenamiento y validación
    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {loss.item():.4f} - Validation Loss: {validation_loss:.4f}")


In [25]:
x=[]

In [28]:
X_patches = torch.split(X, 31, dim=3)
X_patches = [torch.split(patch, 60, dim=2) for patch in X_patches]
X_patches = [item for sublist in X_patches for item in sublist]
X_patches = [torch.split(patch, 60, dim=1) for patch in X_patches]
X_patches = [item for sublist in X_patches for item in sublist]

Y_patches = torch.split(Y, 31, dim=3)
Y_patches = [torch.split(patch, 60, dim=2) for patch in Y_patches]
Y_patches = [item for sublist in Y_patches for item in sublist]
Y_patches = [torch.split(patch, 60, dim=1) for patch in Y_patches]
Y_patches = [item for sublist in Y_patches for item in sublist]

X_patches = torch.stack(X_patches, dim=0)
Y_patches = torch.stack(Y_patches, dim=0)

x.append(X_patches)

In [31]:
x=torch.cat((x), dim=0)

In [32]:
x.shape

torch.Size([160, 3, 60, 60, 31])